In [1]:
import pandas as pd
import FinanceDataReader as fdr
import statsmodels.api as sm
import seaborn as sns

In [4]:
NAVER_STOCK_CODE = '097780'
stock_price = fdr.DataReader(NAVER_STOCK_CODE, '2014', '2022')

In [9]:
stock_pricestock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2014-01-02,8256,8292,7934,8045,126302,-0.021766
2014-01-03,7969,8184,7969,8153,109407,0.013424
2014-01-06,8077,8184,7969,8188,83998,0.004293
2014-01-07,8041,8367,8041,8153,119652,-0.004275
2014-01-08,8292,8331,7969,8009,126036,-0.017662
...,...,...,...,...,...,...
2021-12-24,3438,3467,3377,3385,300848,-0.015702
2021-12-27,3370,3442,3370,3385,388470,0.000000
2021-12-28,3388,3427,3384,3403,324174,0.005318
